# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from requests.utils import requote_uri

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [74]:
cities_df = pd.read_csv("../output_data/cities.csv")
cities_df.head()

,City,Lat,Lng,Max Temp(F),Humidity,Cloudiness,Wind Speed,Country,Date
0,Kapaa,22.0752,-159.3190,73.40,49,40,6.91,US,1612660218
1,Urozhaynoye,44.7883,44.9228,36.27,93,100,7.92,RU,1612660733
2,Ushuaia,-54.8000,-68.3000,50.00,76,75,12.66,AR,1612660266
3,Gravelbourg,49.8834,-106.5512,-14.80,64,97,6.91,CA,1612660733
4,Porto Walter,-8.2686,-72.7439,76.17,88,100,1.25,BR,1612660733


In [3]:
cities_df.shape

(565, 9)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [30]:
#Lets handle Null values
cities_df = cities_df.dropna()

#Fetching location
locations = cities_df[["Lat","Lng"]].astype(float)

#Fetch humidity for weight
Humidity = cities_df["Humidity"].astype(float)
Humidity

0       49.0
1       93.0
2       76.0
3       64.0
4       88.0
       ...  
560     55.0
561     27.0
562    100.0
563     84.0
564     80.0
Name: Humidity, Length: 564, dtype: float64

In [66]:
# Create a poverty Heatmap layer
layout = {
    'width': '960px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '3px',
    'margin': '0 auto 0 auto'}
fig = gmaps.figure(layout=layout,zoom_level=2.15,center=(19,23))
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, dissipating=False, max_intensity=60,point_radius = 2)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='3px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [101]:
Ideal_df = cities_df[(cities_df["Max Temp(F)"]<80)
                     &(cities_df["Max Temp(F)"]>70) 
                     & (cities_df["Wind Speed"]<10)
                     & (cities_df["Cloudiness"] == 0)]
Ideal_df.reset_index(drop=True)

,City,Lat,Lng,Max Temp(F),Humidity,Cloudiness,Wind Speed,Country,Date
0,Hambantota,6.1241,81.1185,71.60,100,0,3.44,LK,1612660284
1,Buin,-33.7333,-70.7500,71.01,82,0,9.22,CL,1612660743
2,Hong Kong,22.2855,114.1577,71.01,62,0,9.28,HK,1612660078
3,Gambēla,8.2500,34.5833,74.01,21,0,1.43,ET,1612660757
4,Santa Cruz de la Sierra,-17.8000,-63.1667,77.00,73,0,5.75,BO,1612660592
5,Benjamín Hill,30.1667,-111.1000,73.00,17,0,6.76,MX,1612660789
6,Yaan,7.3833,8.5667,76.41,21,0,5.08,NG,1612660795
7,Kibala,9.1104,18.3463,75.20,19,0,4.61,TD,1612660796
8,Te Anau,-45.4167,167.7167,75.27,40,0,5.03,NZ,1612660609
9,Paraná,-31.7320,-60.5238,75.99,62,0,7.58,AR,1612660822


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [83]:
hotel_df = Ideal_df.reset_index(drop = True)

#Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] =""
hotel_df.head(3)

,City,Lat,Lng,Max Temp(F),Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Hambantota,6.1241,81.1185,71.60,100,0,3.44,LK,1612660284,
1,Buin,-33.7333,-70.7500,71.01,82,0,9.22,CL,1612660743,
2,Hong Kong,22.2855,114.1577,71.01,62,0,9.28,HK,1612660078,


In [100]:
#Set parameters to search for hotels with 5000 meters.
# geocoordinates
# set up a parameters dictionary
params = {
    "radius": 50000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    #print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


In [88]:
hotel_df

,City,Lat,Lng,Max Temp(F),Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Hambantota,6.1241,81.1185,71.60,100,0,3.44,LK,1612660284,Wild Coast Tented Lodge - Relais & Chateaux
1,Buin,-33.7333,-70.7500,71.01,82,0,9.22,CL,1612660743,VIP Apartments Chile
2,Hong Kong,22.2855,114.1577,71.01,62,0,9.28,HK,1612660078,K11 ARTUS
3,Gambēla,8.2500,34.5833,74.01,21,0,1.43,ET,1612660757,"Grand Resort And Spa,Gambella"
4,Santa Cruz de la Sierra,-17.8000,-63.1667,77.00,73,0,5.75,BO,1612660592,Marriott Santa Cruz de la Sierra Hotel
5,Benjamín Hill,30.1667,-111.1000,73.00,17,0,6.76,MX,1612660789,Hotel La Villa
6,Yaan,7.3833,8.5667,76.41,21,0,5.08,NG,1612660795,The Duchess hotel and Spa
7,Kibala,9.1104,18.3463,75.20,19,0,4.61,TD,1612660796,Auberge Beau Séjour
8,Te Anau,-45.4167,167.7167,75.27,40,0,5.03,NZ,1612660609,Lakefront Lodge Te Anau
9,Paraná,-31.7320,-60.5238,75.99,62,0,7.58,AR,1612660822,InterTower Hotel


In [98]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [99]:
# Add marker layer ontop of heat map
# Create a map using state centroid coordinates to set markers
# Create a marker_layer using the poverty list to fill the info box
fig = gmaps.figure(layout=layout,zoom_level=2.15,center=(19,23))
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='3px', wi…